###Для виконання БЕРТа використаємо бібліотеку ktrain, яка містить в собі багато претренованих моделей

In [ ]:
!pip install ktrain

# Fake/Real news dataset
### Завантажити датасет можна за наступним посиланням:  
> https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset/download

### Загалом датасет має наступний вигляд: 

```Index(['title', 'text', 'subject', 'date'], dtype='object')```

Для подальшої обробки було додано колонку 'Authenticity' та було об'єднано датасети в один news_data.






In [ ]:
import os
import pandas as pd

real = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")

print(real.columns)
fake['Authenticity'] = False
real['Authenticity'] = True
print(real.columns)

news_data = fake.append(real)

news_data = pd.concat([real, fake])

Index(['title', 'text', 'subject', 'date'], dtype='object')
Index(['title', 'text', 'subject', 'date', 'Authenticity'], dtype='object')


In [ ]:
X = np.asarray(news_data["text"])
y = np.asarray(news_data["Authenticity"])

###Ділимо датасет на тренувальний та валідаційний

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

###Далі будемо використовувати модель 'distilbert-base-uncased', яка є зменшеною моделлю BERTa.

In [ ]:
import ktrain
from ktrain import text

MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=["True", "False"])
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

preprocessing train...
language: en
train sequence lengths:
	mean : 404
	95percentile : 901
	99percentile : 1507


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 407
	95percentile : 911
	99percentile : 1509


###Дотреновуємо нашого БЕРТа під специфічну задачу

In [ ]:
learner.fit_onecycle(8e-5, 1)



begin training using onecycle policy with max lr of 8e-05...
5238/5238 [==============================] - 3946s 750ms/step - loss: 0.0560 - accuracy: 0.9825 - val_loss: 0.0026 - val_accuracy: 0.9997


In [ ]:
learner.validate(class_names=["True", "False"])

              precision    recall  f1-score   support

        True       1.00      1.00      1.00      7086
       False       1.00      1.00      1.00      6384

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



array([[7086,    0],
       [   4, 6380]])

In [ ]:
learner.view_top_losses(n=1, preproc=t)

----------
id:10638 | loss:10.04 | true:False | pred:True)



In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.predict(X_test[15])

'False'

In [ ]:
predictor.explain(X_test[15])

/usr/local/lib/python3.7/dist-packages/ktrain/text/predictor.py:115: UserWarning: ktrain requires a forked version of eli5 to support tf.keras. Install with: pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1
  warnings.warn(msg)
